# aule-attention on AMD MI300X

This notebook demonstrates aule-attention running on AMD Instinct MI300X GPU.

**Requirements:**
- PyTorch with ROCm support
- aule-attention (`pip install aule-attention`)

## 1. Setup and Installation

In [ ]:
# Install aule-attention if not already installed
!pip install aule-attention -q

In [ ]:
import torch
import numpy as np
import time

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Import aule-attention and check backends
from aule import flash_attention, get_available_backends, print_backend_info

print("Available backends:", get_available_backends())
print()
print_backend_info()

## 2. Basic Inference

Simple forward pass demonstrating FlashAttention on MI300X.

In [ ]:
# Basic inference example
batch_size = 1
num_heads = 32
seq_len = 2048
head_dim = 128

# Create random tensors (simulating transformer hidden states)
q = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
k = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
v = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)

print(f"Input shapes: Q={q.shape}, K={k.shape}, V={v.shape}")
print(f"Dtype: {q.dtype}")
print(f"Device: {q.device}")

# Run attention
torch.cuda.synchronize()
start = time.perf_counter()

output = flash_attention(q, k, v, causal=True)

torch.cuda.synchronize()
elapsed = time.perf_counter() - start

print(f"\nOutput shape: {output.shape}")
print(f"Time: {elapsed*1000:.2f} ms")

## 3. Numerical Accuracy Test

Compare aule-attention against PyTorch's scaled_dot_product_attention.

In [ ]:
# Accuracy comparison with PyTorch SDPA
torch.manual_seed(42)

batch_size = 2
num_heads = 8
seq_len = 512
head_dim = 64

q = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float32)
k = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float32)
v = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float32)

# aule-attention
out_aule = flash_attention(q, k, v, causal=True)

# PyTorch SDPA
out_torch = torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=True)

# Compare
diff = (out_aule - out_torch).abs()
print(f"Maximum absolute difference: {diff.max().item():.2e}")
print(f"Mean absolute difference: {diff.mean().item():.2e}")
print(f"Relative error: {(diff / out_torch.abs().clamp(min=1e-6)).max().item():.2e}")
print(f"\nOutputs match: {torch.allclose(out_aule, out_torch, atol=1e-4, rtol=1e-4)}")

## 4. Training with Backward Pass

Demonstrate gradient computation for training.

In [ ]:
# Training example with gradients
batch_size = 4
num_heads = 16
seq_len = 1024
head_dim = 64

# Create tensors with gradient tracking
q = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16, requires_grad=True)
k = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16, requires_grad=True)
v = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16, requires_grad=True)

print(f"Input shapes: Q={q.shape}")
print(f"requires_grad: Q={q.requires_grad}, K={k.requires_grad}, V={v.requires_grad}")

# Forward pass
output = flash_attention(q, k, v, causal=True)

# Compute loss (simple sum for demonstration)
loss = output.sum()
print(f"\nLoss: {loss.item():.4f}")

# Backward pass
loss.backward()

print(f"\nGradients computed:")
print(f"  dQ shape: {q.grad.shape}, norm: {q.grad.norm().item():.4f}")
print(f"  dK shape: {k.grad.shape}, norm: {k.grad.norm().item():.4f}")
print(f"  dV shape: {v.grad.shape}, norm: {v.grad.norm().item():.4f}")

## 5. Grouped Query Attention (GQA)

Modern LLMs like Llama 2/3 use GQA where query heads > key/value heads.

In [ ]:
# Grouped Query Attention (Llama-style)
batch_size = 1
num_q_heads = 32   # Query heads
num_kv_heads = 8   # Key/Value heads (4:1 ratio)
seq_len = 2048
head_dim = 128

q = torch.randn(batch_size, num_q_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
k = torch.randn(batch_size, num_kv_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
v = torch.randn(batch_size, num_kv_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)

print(f"GQA Configuration:")
print(f"  Query heads: {num_q_heads}")
print(f"  KV heads: {num_kv_heads}")
print(f"  Ratio: {num_q_heads // num_kv_heads}:1")
print(f"  Sequence length: {seq_len}")

torch.cuda.synchronize()
start = time.perf_counter()

output = flash_attention(q, k, v, causal=True)

torch.cuda.synchronize()
elapsed = time.perf_counter() - start

print(f"\nOutput shape: {output.shape}")
print(f"Time: {elapsed*1000:.2f} ms")

## 6. Performance Benchmark

Measure throughput at different sequence lengths.

In [ ]:
def benchmark_attention(batch_size, num_heads, seq_len, head_dim, num_warmup=5, num_runs=20):
    """Benchmark attention throughput."""
    q = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
    k = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
    v = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
    
    # Warmup
    for _ in range(num_warmup):
        _ = flash_attention(q, k, v, causal=True)
    torch.cuda.synchronize()
    
    # Benchmark
    times = []
    for _ in range(num_runs):
        torch.cuda.synchronize()
        start = time.perf_counter()
        _ = flash_attention(q, k, v, causal=True)
        torch.cuda.synchronize()
        times.append(time.perf_counter() - start)
    
    avg_time = np.mean(times) * 1000  # ms
    
    # Calculate TFLOPS (approximate)
    # FLOPs for attention: 4 * batch * heads * seq^2 * head_dim (for Q@K^T and attn@V)
    flops = 4 * batch_size * num_heads * seq_len * seq_len * head_dim
    tflops = flops / (avg_time / 1000) / 1e12
    
    return avg_time, tflops

print("Performance Benchmark (B=1, H=32, D=128)")
print("=" * 50)
print(f"{'Seq Len':<12} {'Time (ms)':<12} {'TFLOPS':<12}")
print("-" * 50)

for seq_len in [512, 1024, 2048, 4096, 8192]:
    try:
        avg_time, tflops = benchmark_attention(
            batch_size=1,
            num_heads=32,
            seq_len=seq_len,
            head_dim=128
        )
        print(f"{seq_len:<12} {avg_time:<12.2f} {tflops:<12.1f}")
    except Exception as e:
        print(f"{seq_len:<12} Error: {e}")

print("=" * 50)

## 7. Simple Transformer Block

A minimal transformer attention block using aule-attention.

In [ ]:
import torch.nn as nn

class AuleAttentionBlock(nn.Module):
    """Simple attention block using aule-attention."""
    
    def __init__(self, hidden_dim, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        
        self.q_proj = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim, bias=False)
        
    def forward(self, x):
        batch_size, seq_len, hidden_dim = x.shape
        
        # Project to Q, K, V
        q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Attention using aule
        attn_out = flash_attention(q, k, v, causal=True)
        
        # Reshape and project output
        attn_out = attn_out.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        return self.o_proj(attn_out)

# Create and test the block
hidden_dim = 2048
num_heads = 16
batch_size = 2
seq_len = 1024

model = AuleAttentionBlock(hidden_dim, num_heads).cuda().half()
x = torch.randn(batch_size, seq_len, hidden_dim, device='cuda', dtype=torch.float16)

print(f"Model: AuleAttentionBlock")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Num heads: {num_heads}")
print(f"  Head dim: {hidden_dim // num_heads}")
print(f"\nInput shape: {x.shape}")

# Forward pass
torch.cuda.synchronize()
start = time.perf_counter()

output = model(x)

torch.cuda.synchronize()
elapsed = time.perf_counter() - start

print(f"Output shape: {output.shape}")
print(f"Time: {elapsed*1000:.2f} ms")

## 8. Training Loop Example

A minimal training loop with the attention block.

In [ ]:
# Simple training loop
model = AuleAttentionBlock(hidden_dim=1024, num_heads=8).cuda().half()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

batch_size = 4
seq_len = 512
hidden_dim = 1024
num_steps = 10

print("Training loop with aule-attention")
print("=" * 40)

for step in range(num_steps):
    # Generate random input
    x = torch.randn(batch_size, seq_len, hidden_dim, device='cuda', dtype=torch.float16)
    
    # Forward pass
    output = model(x)
    
    # Simple loss (MSE to zero for demonstration)
    loss = output.pow(2).mean()
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Step {step+1:2d}: loss = {loss.item():.6f}")

print("=" * 40)
print("Training complete!")

## 9. Memory Usage

Check GPU memory consumption.

In [ ]:
# Memory usage test
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

batch_size = 1
num_heads = 32
seq_len = 8192
head_dim = 128

q = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
k = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)
v = torch.randn(batch_size, num_heads, seq_len, head_dim, device='cuda', dtype=torch.float16)

input_memory = torch.cuda.memory_allocated() / 1e9
print(f"Memory after input allocation: {input_memory:.2f} GB")

output = flash_attention(q, k, v, causal=True)
torch.cuda.synchronize()

total_memory = torch.cuda.memory_allocated() / 1e9
peak_memory = torch.cuda.max_memory_allocated() / 1e9

print(f"Memory after attention: {total_memory:.2f} GB")
print(f"Peak memory: {peak_memory:.2f} GB")
print(f"\nNote: FlashAttention uses O(N) memory, not O(N^2)")
print(f"Traditional attention at seq_len={seq_len} would need ~{batch_size * num_heads * seq_len * seq_len * 2 / 1e9:.1f} GB for attention matrix alone")

## Summary

aule-attention provides:
- Drop-in FlashAttention for AMD MI300X
- Full training support with backward pass
- GQA/MQA support for modern LLMs
- O(N) memory complexity
- No compilation required at install time

Install: `pip install aule-attention`

Documentation: https://github.com/xenn0010/Aule-Attention